Este algoritmo é projetado para fornecer uma partição fuzzy e um protótipo para cada cluster fuzzy, além de aprender um peso de relevância para cada matriz de dissimilaridade que muda na iteração de cada algoritmo e é diferente de um cluster fuzzy para outro.

In [1]:
# PARÂMETROS INICIAIS

K = 3 # Número de classes
N = 6 # Número de objetos
t = 0 # tempo inicial
m = 1.6 # valor m 
T = 150  # Número máximo de rodadas
e = 1e-10 # Erro permissível
s = 1 # valor s 
q = 1 # cardinalidade q

In [2]:
# Dados de entrada
p1 = 3 # 3 variaveis descritivas para cada objeto
p2 = 4 # 4 variaveis descritivas para cada objeto

# Matrizes de objetos e_i normalziados
E1 = [[0.13043, 0.32609, 0.44022],
     [0.45109, 0.03804, 0.91304],
     [0.4837, 0.34783, 0.47283],
     [0.34783, 0.4837, 1.0],
     [0.49457, 0.0, 0.57609],
     [0.49457, 0.0, 0.40761]]
E2 = [[0.1005, 0.80905, 1.0, 0.8593],
      [0.31658, 0.64824, 0.46231, 0.61809],
      [0.31658, 0.1809, 0.66332, 0.84422],
      [0.79899, 0.08543, 0.78392, 0.78392],
      [0.67839, 0.71357, 0.85427, 0.73367],
      [0.73367, 0.40201, 0.0, 0.1005]]

# Matrizes de Dissimilaridade
D1 = [[0.0, 0.6398, 0.3554, 0.6208, 0.5073, 0.4898],
     [0.6398, 0.0, 0.5392, 0.4656, 0.3418, 0.5087],
     [0.3554, 0.5392, 0.0, 0.5610, 0.3629, 0.3540],
     [0.6208, 0.4656, 0.5610, 0.0, 0.6596, 0.7787],
     [0.5073, 0.3418, 0.3629, 0.6596, 0.0, 0.1684],
     [0.4898, 0.5087, 0.3540, 0.7787, 0.1684, 0.0]]
D2 = [[0.0, 0.64795, 0.74488, 1.03145, 0.61652, 1.46368],
      [0.64795, 0.0, 0.55673, 0.82487, 0.5497, 0.8463],
      [0.74488, 0.55673, 0.0, 0.50992, 0.68068, 1.10271],
      [1.03145, 0.82487, 0.50992, 0.0, 0.64543, 1.08907],
      [0.61652, 0.5497, 0.68068, 0.64543, 0.0, 1.10942],
      [1.46368, 0.8463, 1.10271, 1.08907, 1.10942, 0.0]]
# Dj = D[0] = = D1 e D[1] = D2  Matriz de dissimilaridade com "p" visões
Dj = [[[0.0, 0.6398, 0.3554, 0.6208, 0.5073, 0.4898],
      [0.6398, 0.0, 0.5392, 0.4656, 0.3418, 0.5087],
      [0.3554, 0.5392, 0.0, 0.5610, 0.3629, 0.3540],
      [0.6208, 0.4656, 0.5610, 0.0, 0.6596, 0.7787],
      [0.5073, 0.3418, 0.3629, 0.6596, 0.0, 0.1684],
      [0.4898, 0.5087, 0.3540, 0.7787, 0.1684, 0.0]],
     [[0.0, 0.64795, 0.74488, 1.03145, 0.61652, 1.46368],
      [0.64795, 0.0, 0.55673, 0.82487, 0.5497, 0.8463],
      [0.74488, 0.55673, 0.0, 0.50992, 0.68068, 1.10271],
      [1.03145, 0.82487, 0.50992, 0.0, 0.64543, 1.08907],
      [0.61652, 0.5497, 0.68068, 0.64543, 0.0, 1.10942],
      [1.46368, 0.8463, 1.10271, 1.08907, 1.10942, 0.0]]]
P = len(Dj) # Número de Partições Fuzzy
# Gk é o representante do grupo Pk (1 <= k <= K)
'''G = [[g1 = g11, g12, ..., g1p],
     [g2 = g21, g22, ..., g2p],
     [g3 = g31, g32, ..., g3p],
     [gK = gK1, gK2, ..., gKp]]'''
G = [[2],
     [3],
     [1]]

#Uik é o grau de pertinência exclusivo do objeto i no grupo k. Logo, uik = 1, se o objeto i estiver no grupo k; e 0, otherwise.
#Uik é o representante do grupo Pk (i = 1,..., N) e (k= 1,...,K)
'''U = [[u1 = u11, u12, u13],
     [u2 = u21, u22, u23],
     [u3 = u31, u32, u33],
     [u4 = u41, u42, u43],
     [u5 = u51, u52, u53],
     [u6 = u61, u62, u63]]'''
U = [[1, 0, 0],
     [0, 0, 1],
     [1, 0, 0],
     [0, 0, 1],
     [0, 1, 0],
     [0, 1, 0]]

# V é um vetor k-dimensional de vetores de peso de relevância (um para cada partição fuzzy P)
'''v = [[v1 = v11, v12, ..., v1p],
        [v2 = v21, v22, ..., v2p],
        [v3 = v31, v32, ..., v3p]]'''
v = [[1, 1],
     [1, 1],
     [1, 1]]

In [3]:
# Função de Adequação "J" (Equações: 06 e 07)

def FMatching(ki, ii): # Equação 07
  vkj1 = 0
  for j in range(0,P):
    dj1 = 0
    for e in G[ki]:
      dj1 += Dj[j][ii][e-1]
    vkj1 += dj1 * v[ki][j]
  return vkj1

def fitnessFunction(): # Equação 06
  J = 0
  for i in range(0, N):
    for j in range(0, K):
      J += ((U[i][j])**m) * FMatching(j, i)
  return J

In [4]:
# Etapa 1: Cálculo dos melhores protótipos:
# A partição Fuzzy U e o vetor de vetores de pesos de relevância V_k são fixos

#Procedimento 2.3
import numpy as np # Usar o numpy para utilizar a função "argmin"

def prototype(clt, G_verificação):
  G_out = [] # Para armazenar os prototipos para todos os K cluster
  G_in = []  # Para armazenar os argumentos internos de cada k
  for h in range(0, N):
    G_in.append(argumentos(clt, h))
    #print("G_in: ", G_in)
  L = np.argmin(G_in, axis=0)
  #print("L:", L)
  while len(G_out) == 0:
    if (L+1) not in G_verificação:
      G_out.append(L+1)
    else:
      G_in[L] = 10000
      L = np.argmin(G_in, axis=0)
  return G_out

def argumentos(ki, hi): # Essa função vai retornar o calculo dos somatórios para o k-ésimo cluster e o h-éssimo argumento.
  L = 0
  somaUVD = 0
  for i in range(0, N):
    somaVD = 0
    for j in range(0, P):
       somaVD += ((v[ki][j])**s) * Dj[j][i][hi] # Não ficou tão claro na função, se ele soma também cada matriz de dissimilaridade Dj para as p visões diferentes
    somaUVD += ((U[i][ki])**m) * somaVD
  L += somaUVD
  return L
'''  
G_verificação = []  # Uma lista para armazenar todos os protótipos de G e servir para verificar se o elemento eL já está na lista
for cluster in range(0,K):
  eL = prototype(cluster, G_verificação)
  G_verificação.append(eL)
  print(eL)
  G[cluster] = eL
print(G)'''

'  \nG_verificação = []  # Uma lista para armazenar todos os protótipos de G e servir para verificar se o elemento eL já está na lista\nfor cluster in range(0,K):\n  eL = prototype(cluster, G_verificação)\n  G_verificação.append(eL)\n  print(eL)\n  G[cluster] = eL\nprint(G)'

In [5]:
#Etapa 2: Cálculo do melhor vetor de pesos de relevância das visões nos grupos (Equação 09)
# Partição Fuzzy U e vetor de representantes G fixos!

def relevanceWeights(ki, ji):
  prodUP = 1
  for h in range(0, P):
    uik2 = 0
    for i in range(0,N):
      DjUp2 = 0
      for e in G[ki]:
        DjUp2 += Dj[h][i][(e)-1]
      uik2 += ((U[i][ki])**m) * DjUp2
    prodUP *= uik2

  DjUDown2 = 0
  for i in range(0,N):
    DjDown2 = 0
    for e in G[ki]:
      DjDown2 += Dj[ji][i][(e)-1]
    DjUDown2 += ((U[i][ki])**m) * DjDown2
    
  resultado = (prodUP**(1/P))/(DjUDown2)
  return resultado
'''
for k in range(0, K):
  for j in range(0, P):
    v[k][j] = relevanceWeights(k, j) 
print(v) # A matriz de pesos de relevância depois da etapa 2'''

'\nfor k in range(0, K):\n  for j in range(0, P):\n    v[k][j] = relevanceWeights(k, j) \nprint(v) # A matriz de pesos de relevância depois da etapa 2'

In [6]:
# Etapa 3: Definição da Melhor Partição Fuzzy (Equação 11)
# Vetores de representantes G e vetores dos pesos V são fixos!

def fuzzyPartition(ii, ki, s):
  total3 = 0
  soma3 = 0
  for h in range(0,K):
    vikUP3 = 0
    for j in range(0,P):
      djUP3 = 0
      for e in G[ki]:
        djUP3 += Dj[j][ii][(e)-1]
      vikUP3 += (v[ki][j]**s) * djUP3
    vikDown3 = 0
    for j in range(0,P):
      djDown3 = 0
      for e in G[h]:
        djDown3 += Dj[j][ii][(e)-1]
      vikDown3 += (v[h][j]**s) * djDown3
    if vikDown3 == 0:
      soma3 += 0
    else:
      soma3 += (vikUP3/vikDown3)**(1/(m-1))
  if soma3 == 0:
    total3 = 0
  else:
    total3 = (soma3**(-1))
  return total3
'''
for i in range(0,N):
  for k in range(0,K):
    U[i][k] = fuzzyPartition(i, k, s)
print(U) # A matriz com a Partição Fuzzy depois da etapa 3'''

'\nfor i in range(0,N):\n  for k in range(0,K):\n    U[i][k] = fuzzyPartition(i, k, s)\nprint(U) # A matriz com a Partição Fuzzy depois da etapa 3'

In [7]:
# Algoritmo, união de todas as etapas:

#1) Inicialização
Jt = [10000] # A lista de soluções J inicia com números com diferença bem alta
t = 0 # tempo inicial
T = 11
erro = 1e-03
s = 1
# Para o grau de pertinência Uik inicial.
for i in range(0,N):
  for k in range(0,K):
    U[i][k] = fuzzyPartition(i, k, s)
print(U) # A matriz com a Partição Fuzzy calculada na Inicialização

Jt.append(fitnessFunction()) # Etapa Inicial para o J(0)
print("J('t=0')", Jt[-1])

while (t < T):  # or (abs(Jt[-1]-Jt[-2])>erro)
  t += 1
  #2) Etapa 1:
  G = [] # Armazenar os K clusters designados
  for particao in range(0,K):
    G.append([])
  G_verificação = []  # Uma lista para armazenar todos os protótipos de G e servir para verificar se o elemento eL já está na lista
  for cluster in range(0,K):
    eL = prototype(cluster, G_verificação)
    G_verificação.append(eL)
    G[cluster] = eL
  print(G)
  #3) Etapa 2:
  for k in range(0, K):
    for j in range(0, P):
      v[k][j] = relevanceWeights(k, j) 
  
  #4) Etapa 3:
  for i in range(0,N):
    for k in range(0,K):
      U[i][k] = fuzzyPartition(i, k, s)

  #5) Critério de parada:
  Jt.append(fitnessFunction())
  print("ERRO", abs(Jt[-1]-Jt[-2]))
  print("t = %.f e J(t) = %.10f" %(t, Jt[-1]))
  print("U = ",U)
  print("V = ",v)

#print("J: ",Jt[-1])
#print("U = ",U)
#print("V = ",v)

[[0.4348181366644634, 0.5651818633355365, 0], [0, 0.5668036814162779, 0.4331963185837222], [0.5016505669187326, 0, 0.4983494330812675], [0.3303671550203165, 0.45079767677009935, 0.2188351682095842], [0.40834548013530936, 0.3140656358907923, 0.27758888397389836], [0.41154254054081385, 0.3647749923064576, 0.2236824671527286]]
J('t=0') 4.265570847209421
[[3], [2], [5]]
ERRO 0.4568811109533608
t = 1 e J(t) = 3.8086897363
U =  [[0.397745478221526, 0.26864276106582363, 0.3336117607126504], [0.38423528677992974, 0, 0.6157647132200703], [0, 0.4552358386692843, 0.5447641613307158], [0.3821127360664132, 0.33320421048088317, 0.28468305345270356], [0.4443688229374979, 0.555631177062502, 0], [0.2970060314810203, 0.2836965430663172, 0.4192974254526626]]
V =  [[1.4039378297700102, 0.7122822526719845], [1.1728715790986763, 0.8526082631897998], [1.3647089715348222, 0.7327569620029304]]
[[5], [3], [2]]
ERRO 0.0025338481573360383
t = 2 e J(t) = 3.8112235844
U =  [[0.3388236734540882, 0.3926795685343371, 